In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import umap
from sklearn.manifold import TSNE
from scipy.stats import pearsonr

df = pd.read_csv("./dataset.csv")
df.columns = df.columns.str.lower()
df.columns


In [ ]:
sns.set(font_scale=1.5)
plt.figure(figsize=(10, 10))

class_counts = df["class"].value_counts(normalize=True) * 100
class_labels = [
    f"{label} ({value:.1f}%)"
    for label, value in zip(class_counts.index, class_counts.values)
]

patches, texts = plt.pie(
    x=class_counts,
    labels=class_labels,
    startangle=90,
    labeldistance=0.5,
)

for text in texts:
    text.set_fontweight("bold")
    text.set_horizontalalignment("center")
    text.set_fontsize(20)

plt.axis("equal")
plt.show()


In [ ]:
df


In [ ]:
df.info()


In [ ]:
df.isnull().sum()


In [ ]:
dfn = df.iloc[:, :-1]
dfn.describe().drop(["count"], axis=0).round(3)


In [ ]:
for column in df.columns[:-1]:
    custom_params = {"axes.spines.right": False, "axes.spines.top": False}
    sns.set_theme(style="ticks", rc=custom_params)
    sns.set_context("poster")

    plt.figure(figsize=(8, 5))
    sns.histplot(data=df, x=column, kde=True, hue="class", legend=False)

    plt.xlabel(column.title(), labelpad=10)
    plt.ylabel("Count", labelpad=10)

    plt.savefig(f"./stat-{column}.png", bbox_inches="tight")
    plt.show()


## Normalize


In [ ]:
def normalize(df):
    normed = df.copy()
    numcols = df.select_dtypes(include=["float64", "int64"]).columns
    normed[numcols] = (normed[numcols] - normed[numcols].min()) / (
        normed[numcols].max() - normed[numcols].min()
    )
    return normed


df_normed = normalize(df)


## Outliers


### Boxplot with outliers:


In [ ]:
def create_outlier_boxplot(df):
    custom_params = {"axes.spines.right": False, "axes.spines.top": False}
    sns.set_theme(style="ticks", rc=custom_params)

    columns_to_melt = [
        "area",
        "majoraxislength",
        "minoraxislength",
        "eccentricity",
        "convexarea",
        "extent",
        "perimeter",
    ]

    df_melted = pd.melt(
        df,
        id_vars="class",
        value_vars=columns_to_melt,
        var_name="metric",
        value_name="value",
    )

    plt.figure(figsize=(17.8, 20.6))

    ax = sns.boxplot(data=df_melted, x="value", y="metric", hue="class", fliersize=12)

    ax.tick_params(labelsize=30)

    ax.set_xlabel("Value", fontsize=30, labelpad=15)
    ax.set_ylabel("Metric", fontsize=30, labelpad=15)

    ax.set_xlabel("Value")
    ax.set_ylabel("Metric")

    handles, labels = ax.get_legend_handles_labels()

    ax.legend_.remove()
    ax.legend(
        handles=handles,
        labels=labels,
        loc="center left",
        bbox_to_anchor=(0.99, 0.5),
        fontsize=30,
        frameon=False,
    )

    plt.tight_layout()
    plt.show()


create_outlier_boxplot(df_normed)


### Identify mild/extreme outliers:


In [ ]:
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)


def get_outliers(df: pd.DataFrame, column: str):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    extreme_lower_bound = Q1 - 3 * IQR
    extreme_upper_bound = Q3 + 3 * IQR

    extreme_outliers = df[
        (df[column] < extreme_lower_bound) | (df[column] > extreme_upper_bound)
    ]

    mild_outliers = df[
        ((df[column] < lower_bound) | (df[column] > upper_bound))
        & ~(df[column] < extreme_lower_bound)
        & ~(df[column] > extreme_upper_bound)
    ]

    return (
        {"mild": mild_outliers.shape[0], "extreme": extreme_outliers.shape[0]},
        extreme_outliers.index,
        mild_outliers.index,
    )


def find_outlier_counts(df: pd.DataFrame) -> dict:
    outlier_counts = {}
    for column in df.select_dtypes(include=["number"]).columns:
        outlier_counts[column], _, _ = get_outliers(df, column)
    return outlier_counts


def find_outlier_indices(df: pd.DataFrame):
    outlier_indices = list()
    extreme_outlier_indices = list()

    for column in df.select_dtypes(include=["number"]).columns:
        _, eoi, oi = get_outliers(df, column)
        outlier_indices.extend(oi)
        extreme_outlier_indices.extend(eoi)

    unique_outlier_indices = np.unique(outlier_indices)
    unique_extreme_outlier_indices = np.unique(extreme_outlier_indices)

    return unique_extreme_outlier_indices, unique_outlier_indices


outlier_counts_df = find_outlier_counts(df)

print("Outlier counts:")
for col, counts in outlier_counts_df.items():
    print(f"{col} - Mild: {counts['mild']}, Extreme: {counts['extreme']}")

extreme_outlier_indices, outlier_indices = find_outlier_indices(df)


print(f"\nTotal extreme outlier count: {len(extreme_outlier_indices)}")
print(f"Total mild outlier count: {len(outlier_indices)}\n")


### Remove extreme outliers:


In [ ]:
df_clean: pd.DataFrame = df.drop(extreme_outlier_indices)
df_clean_normed: pd.DataFrame = normalize(df_clean)

_, outlier_indices = find_outlier_indices(df_clean)

print(extreme_outlier_indices)
print(outlier_indices)

create_outlier_boxplot(df_clean_normed)


## Correlations


In [ ]:
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)

df_clean_numeric = df_clean.select_dtypes(include=["float64", "int64"])

correlation_matrix = df_clean_numeric.corr()
correlation_matrix.to_latex("correlation_matrix.tex")
correlation_matrix.round(3).to_latex("correlation_matrix.tex", float_format="%.3f")

mask = np.triu(np.ones_like(correlation_matrix, dtype=bool), 1)

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", mask=mask)
plt.xticks(rotation=45)
plt.show()

pvalues = df_clean_numeric.corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(
    len(df_clean_numeric.columns)
)

pvalues.round(3).to_latex("pvalues.tex", float_format="%.3f")


## Dimensionality reduction visualization:


In [ ]:
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)

sns.set_theme(style="ticks")
plt.rcParams.update({"font.size": 25})
sns.set_context("talk")


def graph_algo(results, filename):
    local_df = pd.DataFrame(results, columns=["comp1", "comp2"], index=df_clean.index)
    local_df["class"] = df_clean["class"].values
    local_df["type"] = "inlier"
    local_df.loc[list(outlier_indices), "type"] = "outlier"

    plt.figure(figsize=(12, 8))

    d = {"inlier": 50, "outlier": 200}
    local_df["size_outliers"] = local_df["type"].map(d)

    ax = sns.scatterplot(
        x="comp1",
        y="comp2",
        hue="class",
        style="type",
        data=local_df,
    )
    ax.set(xlabel=None, ylabel=None)
    ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)

    plt.savefig(filename, bbox_inches="tight")

    plt.show()


## t-SNE


In [ ]:
def tsne_results(perplexity, df_clean):
    tsne = TSNE(n_components=2, random_state=42, perplexity=perplexity)
    tsne_results = tsne.fit_transform(df_clean.drop(columns="class"))
    return tsne_results


In [ ]:
def graph_tsne(perplexity, df_clean, filename):
    res = tsne_results(perplexity, df_clean)
    graph_algo(res, filename)


### t-SNE (original dataset):


In [ ]:
for perplexity in [10, 30, 50]:
    filename = f"./t-sne-df-{perplexity}.png"
    graph_tsne(perplexity, df_clean, filename)


### t-SNE (normalized dataset):


In [ ]:
for perplexity in [10, 30, 50]:
    filename = f"./t-sne-norm-df-{perplexity}.png"
    graph_tsne(perplexity, df_clean_normed, filename)


## Umap


In [ ]:
def umap_results(n_neighbors, dataset):
    reducer = umap.UMAP(n_jobs=1, random_state=42, n_neighbors=n_neighbors)
    umap_results = reducer.fit_transform(dataset.drop(columns=["class"]))
    return umap_results


In [ ]:
def graph_umap(n_neighbors, dataset, filename) -> None:
    res = umap_results(n_neighbors, dataset)
    graph_algo(res, filename)


### Umap (original dataset):


In [ ]:
for n_neighbors in [10, 75, 150]:
    filename = f"./umap-df-{perplexity}.png"
    graph_umap(n_neighbors, df_clean, filename)


### Umap (normalized dataset):


In [ ]:
for n_neighbors in [10, 75, 150]:
    filename = f"./umap-norm-df-{perplexity}.png"
    graph_umap(n_neighbors, df_clean_normed, filename)


## Umap vs t-SNE (Silhouette Score)


In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
import umap
from sklearn.manifold import TSNE


# Function to compute t-SNE embeddings
def tsne_embeddings(data, perplexity):
    tsne = TSNE(n_components=2, random_state=42, perplexity=perplexity)
    return tsne.fit_transform(data.drop(columns="class"))


# Function to compute UMAP embeddings
def umap_embeddings(data, neighbors):
    reducer = umap.UMAP(n_jobs=1, random_state=42, n_neighbors=neighbors)
    return reducer.fit_transform(data.drop(columns=["class"]))


# Function to compute Silhouette Score
def compute_silhouette_score(embeddings):
    kmeans = KMeans(n_clusters=2, random_state=42)
    cluster_labels = kmeans.fit_predict(embeddings)
    score = silhouette_score(embeddings, cluster_labels)
    return score


tsne_scores = []
umap_scores = []

for value in [10, 30, 50]:
    tsne_embedding = tsne_embeddings(df_clean_normed, value)

    tsne_scores.append(compute_silhouette_score(tsne_embedding))

for value in [10, 75, 150]:
    umap_embedding = umap_embeddings(df_clean_normed, value)

    umap_scores.append(compute_silhouette_score(umap_embedding))


print("t-SNE Silhouette scores:", tsne_scores)
print("UMAP Silhouette scores:", umap_scores)
